# urgent satd characterization process

## 1.随机从 output_file_gai.csv 文件中，分别抽取 50 条 surive_time=0 和 surive_time=1 的数据，且保证 function_before_tokenized 和 function_after_tokenized 字段不为空。

In [1]:
import pandas as pd
import random

# 读取CSV文件
output_df = pd.read_csv('output_file_gai.csv')

# 过滤掉 `function_before_tokenized` 和 `function_after_tokenized` 为空的行
filtered_df = output_df[(output_df['function_before_tokenized'].notna()) & (output_df['function_after_tokenized'].notna())]

# 分别提取 `surive_time=0` 和 `surive_time=1` 的数据
surive_time_0 = filtered_df[filtered_df['survive_time'] == 0]
surive_time_1 = filtered_df[filtered_df['survive_time'] >= 1]

# 随机抽取50条数据
sample_0 = surive_time_0.sample(n=50, random_state=42)  # random_state 是为了保证可重复性
sample_1 = surive_time_1.sample(n=50, random_state=42)

# 保存抽取的数据到文件（可选）
sample_0.to_csv('sample_0.csv', index=False)
sample_1.to_csv('sample_1.csv', index=False)

print("Sample extraction complete.")

Sample extraction complete.


## 2.提取特征
<img src="特征维度.png" width=1200>

- Churn Workload：改动的代码量，根据token化之后比较function_before and function_after的相似度，然后显著性检验 
- Commit Goal: SATD注释是否清晰，目标明确
- Working on release: 离上一次发布的版本的时间
- Project Startup: 离项目刚开始的时间
- Commit workload: 提交次数，代表开发人员做出的不同贡献的数量
- Tenure：作者关于这个项目的经验

### Churn Workload

- 读取CSV文件：从 sample_0.csv 和 sample_1.csv 文件中读取数据。
- 相似度计算函数：使用 difflib.SequenceMatcher 函数来计算两个字符串的相似度。该函数返回值介于 0 到 1 之间，1 表示完全相同，0 表示完全不同。
- 应用相似度计算：使用 apply() 方法逐行应用相似度计算，并将结果存储到新的 similarity 列中。
- 保存结果：将包含相似度的结果保存到新的 CSV 文件中（sample_0_with_similarity.csv 和 sample_1_with_similarity.csv）。

In [2]:
import pandas as pd
import difflib

# 读取CSV文件
sample_0 = pd.read_csv('sample_0.csv')
sample_1 = pd.read_csv('sample_1.csv')

# 定义计算相似度的函数
def calculate_similarity(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()

# 计算 sample_0 的相似度
sample_0['similarity'] = sample_0.apply(lambda row: calculate_similarity(row['function_before_tokenized'], row['function_after_tokenized']), axis=1)

# 计算 sample_1 的相似度
sample_1['similarity'] = sample_1.apply(lambda row: calculate_similarity(row['function_before_tokenized'], row['function_after_tokenized']), axis=1)

# 保存结果到新文件
sample_0.to_csv('sample_0_with_similarity.csv', index=False)
sample_1.to_csv('sample_1_with_similarity.csv', index=False)

print("Similarity calculation complete.")

Similarity calculation complete.


- 读取 CSV 文件：使用 pandas 库读取 sample_0.csv 和 sample_1.csv 文件。
- 获取相似度数据：假设你已经计算了相似度，并将相似度存储在 similarity 列中。我们通过 dropna() 方法确保没有缺失值。
- t 检验：使用 scipy.stats.ttest_ind() 函数对 survive_time=0 和 survive_time=1 的相似度数据进行独立样本 t 检验。
- 显著性检验：根据 p 值判断是否有显著性差异，通常使用的显著性水平为 0.05。

In [3]:
from scipy import stats

# 读取 CSV 文件
sample_0 = pd.read_csv('sample_0_with_similarity.csv')
sample_1 = pd.read_csv('sample_1_with_similarity.csv')

# 假设相似度数据存储在 "similarity" 列
survive_time_0 = sample_0['similarity'].dropna()  # 确保数据中没有缺失值
survive_time_1 = sample_1['similarity'].dropna()

# 进行独立样本 t 检验
t_statistic, p_value = stats.ttest_ind(survive_time_0, survive_time_1)

# 输出结果
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# 检查显著性水平 (通常使用 alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print("两个数据集之间存在显著差异。")
else:
    print("两个数据集之间不存在显著差异。")

T-statistic: -2.1048953643854884
P-value: 0.03785823687703119
两个数据集之间存在显著差异。


> 结果：
> T-statistic: -2.1048953643854884
> P-value: 0.03785823687703119
> 两个数据集之间存在显著差异。

计算平均值和中位数

In [4]:
import pandas as pd

# 读取 CSV 文件
sample_0 = pd.read_csv('sample_0_with_similarity.csv')
sample_1 = pd.read_csv('sample_1_with_similarity.csv')

# 假设相似度数据存储在 "similarity" 列
survive_time_0 = sample_0['similarity'].dropna()  # 确保没有缺失值
survive_time_1 = sample_1['similarity'].dropna()

# 计算平均值和中位数
mean_0 = survive_time_0.mean()
median_0 = survive_time_0.median()

mean_1 = survive_time_1.mean()
median_1 = survive_time_1.median()

# 输出结果
print(f"Survive_time=0 的平均值: {mean_0}")
print(f"Survive_time=0 的中位数: {median_0}")

print(f"Survive_time=1 的平均值: {mean_1}")
print(f"Survive_time=1 的中位数: {median_1}")

Survive_time=0 的平均值: 0.6833865717347414
Survive_time=0 的中位数: 0.7542094756904894
Survive_time=1 的平均值: 0.7800271740024162
Survive_time=1 的中位数: 0.868339775028901


### Commit Goal

- 计算单词数量：使用 apply(lambda x: len(str(x).split())) 计算每条 comment 的单词数量。
- 保存结果：将包含单词数量的结果保存到新的 CSV 文件中，分别为 sample_0_with_word_count.csv 和 sample_1_with_word_count.csv（这一步是可选的）。
- 统计单词数量：对两个数据集中的单词数量分别计算平均值和中位数。
- 显著性检验：通过 t 检验分析两个数据集中 comment_word_count 是否存在显著差异。

In [5]:
import pandas as pd
from scipy import stats

# 读取 CSV 文件
sample_0 = pd.read_csv('sample_0.csv')
sample_1 = pd.read_csv('sample_1.csv')

# 假设 comment 数据存储在 "comment" 列，计算单词数量
sample_0['comment_word_count'] = sample_0['comment'].apply(lambda x: len(str(x).split()))
sample_1['comment_word_count'] = sample_1['comment'].apply(lambda x: len(str(x).split()))

# 保存带有 comment_word_count 列的新文件（可选）
sample_0.to_csv('sample_0_with_word_count.csv', index=False)
sample_1.to_csv('sample_1_with_word_count.csv', index=False)

# 提取 comment_word_count 列
comment_word_count_0 = sample_0['comment_word_count'].dropna()  # 确保没有缺失值
comment_word_count_1 = sample_1['comment_word_count'].dropna()

# 计算平均值和中位数
mean_word_count_0 = comment_word_count_0.mean()
median_word_count_0 = comment_word_count_0.median()

mean_word_count_1 = comment_word_count_1.mean()
median_word_count_1 = comment_word_count_1.median()

# 输出平均值和中位数
print(f"Survive_time=0 的评论平均单词数: {mean_word_count_0}")
print(f"Survive_time=0 的评论中位数单词数: {median_word_count_0}")

print(f"Survive_time=1 的评论平均单词数: {mean_word_count_1}")
print(f"Survive_time=1 的评论中位数单词数: {median_word_count_1}")

# 进行独立样本 t 检验
t_statistic, p_value = stats.ttest_ind(comment_word_count_0, comment_word_count_1)

# 输出 t 检验结果
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# 检查显著性水平 (通常使用 alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print("两个数据集之间存在显著差异。")
else:
    print("两个数据集之间不存在显著差异。")

Survive_time=0 的评论平均单词数: 8.06
Survive_time=0 的评论中位数单词数: 7.0
Survive_time=1 的评论平均单词数: 8.06
Survive_time=1 的评论中位数单词数: 7.0
T-statistic: 0.0
P-value: 1.0
两个数据集之间不存在显著差异。


- 使用LLM，星火认知大模型，`isClear()` 判别SATD的目标是否明确

In [ ]:
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage
import time


def isClear(comment):
    #星火认知大模型Spark Max的URL值，其他版本大模型URL值请前往文档（https://www.xfyun.cn/doc/spark/Web.html）查看
    SPARKAI_URL = 'wss://spark-api.xf-yun.com/v1.1/chat'
    #星火认知大模型调用秘钥信息，请前往讯飞开放平台控制台（https://console.xfyun.cn/services/bm35）查看
    SPARKAI_APP_ID = 'c3ad6fa7'
    SPARKAI_API_SECRET = 'MmFhNmY4ZDZiZDEwMjkzYmUxZDc5NTlh'
    SPARKAI_API_KEY = '3da9a6c9584eb9db8959b4e651ff1322'
    #星火认知大模型Spark Max的domain值，其他版本大模型domain值请前往文档（https://www.xfyun.cn/doc/spark/Web.html）查看
    SPARKAI_DOMAIN = 'general'
    spark = ChatSparkLLM(
        spark_api_url=SPARKAI_URL,
        spark_app_id=SPARKAI_APP_ID,
        spark_api_key=SPARKAI_API_KEY,
        spark_api_secret=SPARKAI_API_SECRET,
        spark_llm_domain=SPARKAI_DOMAIN,
        streaming=False,
    )
    """
        判断SATD注释是否清晰，目标明确。
        
        Args:
        comment: SATD注释字符串。
        
        Returns:
        True or False。
    """
    messages = [ChatMessage(
        role="user",
        content=f'你是一个程序员，你认为注释{comment}的目的含义是否清晰, 你只需要回答Yes or No'
    )]
    handler = ChunkPrintHandler()
    # time.sleep(1)  # 休眠
    answer = spark.generate([messages], callbacks=[handler]).generations[0][0].text
    return len(answer) >= 10 or answer == "Yes"

分别对sample.csv文件进行判定

In [ ]:
# 读取csv文件
df_0 = pd.read_csv('sample_0.csv')  # 将'sample_0.csv'替换为你的文件名
df_1 = pd.read_csv('sample_1.csv')  # 将'sample_1.csv'替换为你的文件名

# 对comment列进行判定
df_0['isClear'] = df_0['comment'].apply(isClear)
df_1['isClear'] = df_1['comment'].apply(isClear)

# 打印结果
print("sample_0.csv:")
print(df_0[['comment', 'isClear']])

print("\nsample_1.csv:")
print(df_1[['comment', 'isClear']])

# 保存结果
df_0.to_csv('sample_0.csv', index=False)
df_1.to_csv('sample_1.csv', index=False)

对 sample_0.csv 和 sample_1.csv 中的 isClear 列进行显著性统计

看看 survive_time 为 0 和 1 的两组样本，它们的技术债务注释清晰度的比例是否有显著差异。

由于 isClear 是一个二元变量（True 或 False），我们可以使用 **卡方检验** 来进行显著性统计。

In [ ]:
from scipy.stats import chi2_contingency

# 读取csv文件
df_0 = pd.read_csv('sample_0.csv')  # 将'sample_0_with_isClear.csv'替换为你的文件名
df_1 = pd.read_csv('sample_1.csv')  # 将'sample_1_with_isClear.csv'替换为你的文件名

# 统计 isClear 的数量
count_0 = df_0['isClear'].value_counts()
count_1 = df_1['isClear'].value_counts()

# 构建列联表
contingency_table = pd.DataFrame({
    'survive_time_0': count_0,
    'survive_time_1': count_1
}).fillna(0)  # 用0填充缺失值

# 进行卡方检验
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# 打印结果
print('卡方检验结果:')
print(f'卡方值: {chi2}')
print(f'p值: {p_value}')
print(f'自由度: {dof}')
print('期望频数:\n', expected)

# 解释p值
alpha = 0.05  # 设置显著性水平
if p_value < alpha:
    print('拒绝原假设，两组样本的 isClear 比例存在显著差异。')
else:
    print('无法拒绝原假设，两组样本的 isClear 比例不存在显著差异。')